In [1]:
import os
import pandas as pd

In [2]:
cwd = os.getcwd()
files = []
for x in os.listdir(cwd + '/Games'):
    if x != '.DS_Store':
        files.append('Games/' + x)

In [3]:
dummy = pd.read_csv(files[0])
index = list(dummy.transpose().iloc[0].values)
values = list(dummy.transpose().iloc[1].values)
values = [[x] for x in values]
dictionary = dict(zip(index, values))
temp_df = pd.DataFrame.from_dict(dictionary) 

In [4]:
for i in range(1,len(files)):
    dummy = pd.read_csv(files[i])
    values = list(dummy.transpose().iloc[1].values)
    dummy = pd.DataFrame(values).transpose()
    dummy.columns = index
    temp_df = temp_df.append(dummy)


In [5]:
def replace(team):
    team = team.replace('cha', 'cho')
    team = team.replace('sea', 'okc')
    team = team.replace('njn', 'brk')
    team = team.replace('noh','nop')
    return team

In [6]:
temp_df['Code'] = temp_df['Code'].apply(replace)

In [7]:
odds_df = pd.read_csv('oddsdf.csv', encoding = "ISO-8859-1")

In [8]:
def two_digits(x):
    if int(x)<10:
        return str(0) + str(x)
    else:
        return str(x)

In [9]:
temp_df['Date'] = temp_df['Year'].astype(str) + temp_df['Month'].astype(str) + temp_df['Day'].astype(str)
odds_df['Date'] = odds_df['Year'].astype(str) + odds_df['Month'].apply(two_digits) + odds_df['Day'].apply(two_digits)
dates = list(set(temp_df['Date'].values))

In [10]:
home_df = temp_df[temp_df['Home']=='1']
away_df = temp_df[temp_df['Home']=='0']

In [11]:
home_df = home_df.sort_values(['Year', 'Month', 'Day', 'Code'])
away_df = away_df.sort_values(['Year', 'Month', 'Day', 'Code'])
home_df.to_csv('Games/home_df.csv')
away_df.to_csv('Games/away_df.csv')

In [12]:
odds_df = odds_df[odds_df['Date'].isin(dates)]

In [13]:
odds_df['Spread'] = odds_df['Odds'].str[:-4]
odds_df['Line'] = odds_df['Odds'].str[-4:]
teams = list(set(odds_df['Team']))
teams.sort()
for y in ['Baltimore', 'Pittsburgh']:
    teams.remove(y)
abreviations = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU',
         'IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO',
         'POR','SAC','SAS','TOR','UTA','WAS']
abr = dict(zip(teams, abreviations)) # zip the team names from the odds df and the abreviations for the website url
odds_df = odds_df[odds_df['Team'].isin(teams)]
all_teams = list(odds_df['Team'])
all_teams = [abr[x] for x in all_teams]
odds_df['Abreviation'] = all_teams

In [14]:
home_odds = odds_df[odds_df['Unnamed: 0']%2 == 1]
away_odds = odds_df[odds_df['Unnamed: 0']%2 == 0]

In [15]:
codes = []
for i in range(len(home_odds)):
    home = home_odds.iloc[i]['Abreviation']
    away = away_odds.iloc[i]['Abreviation']
    codes.append((home+away).lower())

In [16]:
home_odds['Code'] = codes
away_odds['Code'] = codes

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
#only include games that have game stats and odds
checkerlst = list(set(list((temp_df['Date'] + temp_df['Code']).tolist())).intersection(list((home_odds['Date'] + home_odds['Code']).tolist())))
home_odds['Checker'] = home_odds['Date'] + home_odds['Code']
away_odds['Checker'] = away_odds['Date'] + away_odds['Code']
home_df['Checker'] = home_df['Date'] + home_df['Code']
away_df['Checker'] = away_df['Date'] + away_df['Code']
home_odds = home_odds[home_odds['Checker'].isin(checkerlst)]
away_odds = away_odds[away_odds['Checker'].isin(checkerlst)]
home_df = home_df[home_df['Checker'].isin(checkerlst)]
away_df = away_df[away_df['Checker'].isin(checkerlst)]

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
home_odds = home_odds[['Team', 'Spread', 'Line', 'Checker']]
away_odds = away_odds[['Team', 'Spread', 'Line', 'Checker']]

In [19]:
home_completed = pd.merge(home_df, home_odds)
away_completed = pd.merge(away_df, away_odds)

In [20]:
checkerlst = list(set(home_completed['Checker'].tolist()).intersection(away_completed['Checker'].tolist()))
home_completed = home_completed[home_completed['Checker'].isin(checkerlst)]
away_completed = away_completed[away_completed['Checker'].isin(checkerlst)]

In [21]:
home_completed.to_csv('home_completed.csv')
away_completed.to_csv('away_completed.csv')

In [22]:
away_completed

,+/-,3P,3P%,3PA,3PAr,AST,BLK,Code,DRB,Day,...,TOV,TS%,Wins,Year,eFG%,Date,Checker,Team,Spread,Line
0,105,5,0.45454545454545453,11,0.05555555555555555,24,7,gswuta,40,30,...,19,0.5527210884353742,0,2007,0.48333333333333334,20071030,20071030gswuta,Utah,+4,-110
1,10,6,0.2727272727272727,22,0.08108108108108109,23,5,lalhou,37,30,...,18,0.5419899589228663,0,2007,0.5,20071030,20071030lalhou,Houston,-3,-110
2,-45,6,0.46153846153846156,13,0.07692307692307693,15,4,saspor,32,30,...,16,0.5673841834347215,0,2007,0.5384615384615384,20071030,20071030saspor,Portland,+12.5,-110
3,90,9,0.5,18,0.13043478260869565,23,8,cledal,31,31,...,16,0.5912596401028278,0,2007,0.5434782608695652,20071031,20071031cledal,Dallas,-4,-110
4,-45,6,0.3,20,0.06060606060606061,15,6,indwas,33,31,...,18,0.46296296296296297,0,2007,0.3939393939393939,20071031,20071031indwas,Washington,+1,-110
5,15,12,0.5217391304347826,23,0.13793103448275862,23,5,memsas,35,31,...,9,0.5378568473314025,1,2007,0.5057471264367817,20071031,20071031memsas,San Antonio,-7,-110
6,-95,4,0.23529411764705882,17,0.04597701149425287,16,4,orlmil,30,31,...,14,0.42925113777410007,0,2007,0.40229885057471265,20071031,20071031orlmil,Milwaukee,+7.5,-110
7,-45,7,0.4117647058823529,17,0.08536585365853659,22,9,torphi,31,31,...,17,0.5264871906209292,0,2007,0.5060975609756098,20071031,20071031torphi,Philadelphia,+6.5,-110
8,55,6,0.3157894736842105,19,0.075,19,4,miadet,32,01,...,10,0.48819742489270385,0,2007,0.425,20071101,20071101miadet,Detroit,+4.5,-110
9,55,5,0.3125,16,0.0625,19,8,utahou,32,01,...,14,0.5686695278969957,1,2007,0.51875,20071101,20071101utahou,Houston,+4,-110


In [23]:
main = home_completed.append(away_completed, ignore_index=True)

In [24]:
main = main[main['Team']=='Philadelphia']

In [25]:
main

,+/-,3P,3P%,3PA,3PAr,AST,BLK,Code,DRB,Day,...,TOV,TS%,Wins,Year,eFG%,Date,Checker,Team,Spread,Line
24,-25,3,0.2,15,0.037037037037037035,15,4,phibrk,32,03,...,14,0.4627681952040387,1,2007,0.41358024691358025,20071103,20071103phibrk,Philadelphia,+2.5,-110
45,155,5,0.45454545454545453,11,0.06944444444444445,18,7,phicho,29,07,...,20,0.546003717472119,1,2007,0.4930555555555556,20071107,20071107phicho,Philadelphia,-3,-110
58,-10,7,0.3333333333333333,21,0.07954545454545454,22,4,phitor,27,09,...,11,0.5248675091724418,2,2007,0.4943181818181818,20071109,20071109phitor,Philadelphia,-1,-110
71,-105,2,0.13333333333333333,15,0.02702702702702703,21,2,phinop,30,11,...,18,0.44665012406947896,2,2007,0.4189189189189189,20071111,20071111phinop,Philadelphia,+2.5,-110
100,20,3,0.375,8,0.0410958904109589,19,4,phipor,32,16,...,21,0.5653883972468043,2,2007,0.5273972602739726,20071116,20071116phipor,Philadelphia,-2.5,-110
155,-10,5,0.45454545454545453,11,0.06172839506172839,22,8,phigsw,35,24,...,16,0.501432664756447,3,2007,0.4506172839506173,20071124,20071124phigsw,Philadelphia,+2,-110
174,-55,5,0.3125,16,0.06493506493506493,20,7,phiuta,23,28,...,16,0.5165289256198347,4,2007,0.461038961038961,20071128,20071128phiuta,Philadelphia,+7.5,-110
189,5,0,0.0,14,0.0,24,6,phiwas,29,30,...,8,0.4406885109912899,4,2007,0.38823529411764707,20071130,20071130phiwas,Philadelphia,-1.5,-110
208,-45,2,0.18181818181818182,11,0.02666666666666667,14,5,phiatl,36,03,...,17,0.45930232558139533,5,2007,0.41333333333333333,20071203,20071203phiatl,Philadelphia,-2,-110
220,-50,4,0.2857142857142857,14,0.05063291139240506,22,2,phibos,21,05,...,13,0.598558809855881,5,2007,0.5822784810126582,20071205,20071205phibos,Philadelphia,+9,-110
